# Examen Data science 1
## 2023 - P3

Naam: ...
Klas: INF...
studentnummer: ...

In [2]:
# zet hier al je imports:
import numpy as np
import pandas as pd
import math
import mathplotlib.pyplot as plt
from scipy import stats
import csv

In [3]:
# zet hier al je zelfgeschreven functies:
## data management ##

def fill_list(list):
    temp = []
    for i in range(len(list)):
        for j in range(list[i][1]):
            temp.append(list[i][0])
    output = pd.Series(temp)
    return output


missing_values = ['n/a', 'na', 'nan', 'N/A', 'NA', 'NaN', 'NAN', '--', 'Missing', 'missing', 'MISSING', 'Unknown',
                  'Weet ik niet', 'None']

percentiles_input = []

for i in np.arange(0, 1, 0.01):
    percentiles_input.append(i)


def inlezen_bestand(filename, delim, decimal):
    df = pd.read_csv(filename, na_values=missing_values, sep=delim, decimal=decimal)
    return df


def nb_of_fields(name_csv_file, delim):
    fields = []
    with open(name_csv_file, newline='') as csvfile:
        data = csv.reader(csvfile, delimiter=delim)
        fields = next(data)
        min_counter = len(fields)
        max_counter = min_counter
        for row in data:
            nb_fields = len(row)
        if nb_fields > max_counter:
            max_counter = nb_fields
        elif nb_fields < min_counter:
            min_counter = nb_fields


def overzicht_na_values(list):
    totaal_aantal_na = list.isna().sum().sum()
    rijen_na = list[list.isna().any(axis=1)]
    kolommen_na = list[list.columns[list.isna().any(axis=0)]]
    print(
        'totaal # NA values: {}\nrijen met NA values: {}\nkolommen met NA values: {}'.format(totaal_aantal_na, rijen_na,
                                                                                             kolommen_na))


### frequenties ###
def alle_freq(x, dropna_boolean):
    t_abs = x.value_counts(dropna=dropna_boolean).sort_index()
    t_rel = (x.value_counts(dropna=dropna_boolean, normalize=True).sort_index() * 100).round(1)
    t_abs_cum = x.value_counts(dropna=dropna_boolean).sort_index().cumsum()
    t_rel_cum = (x.value_counts(dropna=dropna_boolean, normalize=True).sort_index().cumsum() * 100).round(1)
    return pd.DataFrame({'abs freq': t_abs, 'rel freq': t_rel, 'abs cum freq': t_abs_cum, 'rel cum freq': t_rel_cum})


def aantal_klassen(list):
    n = len(list)
    sturges = math.ceil(1 + math.log2(n))
    scott = math.ceil((list.max() - list.min()) / (3.5 * list.std() / (n ** (1 / 3))))
    sqrt = math.ceil(math.sqrt(n))
    print('sturges:{}\nscott:{}\nsqrt:{}'.format(sturges, scott, sqrt))


def klassen_maken(cutpoints, list):
    klassen = pd.cut(list, bins=cutpoints, right=True)
    return klassen


## grafieken ##

def pie_chart(list, title):
    x = list.value_counts().sort_index()
    l = list.dropna()
    l = l.unique()
    l = np.sort(l)
    plt.figure()
    plt.pie(x, labels=l)
    plt.title(title)
    plt.show()


def bar_chart(list, title, x_label, y_label):
    x = list.value_counts().sort_index()
    l = range(0, 30000, 1000)
    l = np.sort(l)
    plt.figure()
    plt.bar(l, x)
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()


def histogram(breekpunten, list, title):
    plt.figure()
    list.plot.hist(bins=breekpunten, title=title)
    plt.show()


def make_boxplot(list):
    plt.figure()
    data_to_plot = list.dropna()
    data_to_plot.plot.box(grid="True")
    plt.show()


def make_boxpplot_zscore(list, column1, column2):
    z = list[[column1, column2]].dropna()
    z.column1 = stats.zscore(z[column1])
    z.column2 = stats.zscore(z[column2])
    plt.figure()
    data_to_plot = [column1, column2]
    z.boxplot(data_to_plot)
    plt.show()


def make_more_boxplots(dict):
    plt.figure()
    data_to_plot = dict
    data_to_plot.plot.box(grid="True")
    plt.show()


## centrummaten ##
def centrum(list):
    modus = list.mode()
    median = list.median()
    average = list.mean()
    h_avg = stats.hmean(list)
    m_avg = stats.gmean(list)
    print('modus: {}\nmedian: {}\naverage: {}\nh_avg {}\nm_avg: {}'.format(modus, median, average, h_avg, m_avg))


def median_categorical(data):
    data_no_na = data.dropna()
    n = len(data_no_na)
    middle = math.floor(n / 2)
    return data_no_na.sort_values().reset_index(drop=True)[middle]


## spreidingsmaten ##
def spreidingsmaten(list):
    variantie = list.dropna().var()
    std_afw = list.dropna().std()
    mean_abs_deviation = list.mad()
    iqr = stats.iqr(list.dropna())
    quantile = list.quantile(q=[0.25, 0.5, 0.75])
    percentiles = list.quantile(q=percentiles_input)
    bereik = max(list) - min(list)
    # standaardscores =  stats.zscore(list.dropna())
    print('variantie: {}\nstd_afw: {}\nmean_abs_deviation: {}\niqr: {}\n{}\n{}\nbereik: {}'.format(variantie, std_afw,
                                                                                                   mean_abs_deviation,
                                                                                                   iqr, quantile,
                                                                                                   percentiles, bereik))


def bepaal_uitschieters(list):
    uitschieters = []
    extreme_uitschieters = []
    quantile = list.quantile(q=[0.25, 0.5, 0.75])
    iqr = quantile.iloc[2] - quantile.iloc[0]
    for i in list:
        if (i < quantile.iloc[0] - (iqr * 1.5) or i > quantile.iloc[2] + (iqr * 1.5)):
            uitschieters.append(i)

    print('{} uitschieters vs {} items in totaal'.format(len(uitschieters), len(list)))

def vergelijken_met_steekproef(waarde, lijst):
    return (waarde - lijst.mean()) / lijst.std()

# DEEL 1

## Vraag 1

antwoorden:
1a)
1b)
1c) zie code
1d) zie code
1e)

In [4]:
# code


## Vraag 2

antwoorden:
2a) zie code
2b)

In [5]:
# code


## Vraag 3

antwoorden:
3a) zie code
3b)
3c)

In [6]:
# code


## Vraag 4

antwoorden:
4a)
4b)

## Vraag 5

antwoorden:
5a) zie code
5b)
5c)
5d)

In [7]:
# code


# DEEL 2

In [8]:
examen_deel2 = pd.read_csv('examen_2023_P3_DEEL2.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'examen_2023_P3_DEEL2.csv'

## Vraag 6

antwoorden:
6a)
6b)
6c)
6d)
6e)
6f)
6g)
6h)

In [ ]:
# code


## Vraag 7

antwoorden:
7a)
7b)
7c)
7d)

In [ ]:
# code


## Vraag 8

antwoorden:
8a)
8b)
8c)
8d)
8e)

In [ ]:
# code


## Vraag 9

antwoorden:
9a)
9b)
9c)
9d)

In [ ]:
# code
